In [1]:
import pandas as pd

In [67]:
%store -r

In [143]:
import os

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import googleapiclient.discovery
import googleapiclient.errors
import google_auth_oauthlib.flow

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

flow = InstalledAppFlow.from_client_secrets_file(
 "Youtube_Oauth.json", scopes = scopes)

flow.run_local_server(port = 8080)

credentials = flow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725695838850-3pp8b4qjgqebrq8iqdag2pmkhcmp7h3i.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=pBQxLOvBIix5ufOb5UT927WZFlElbl&access_type=offline


In [144]:
api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "Youtube_Oauth.json"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

# Trending Video Daily Update Function

In [157]:
# A function update trending video daily
#Parameter:
    #df: current dataset
    #response: the first page of trending video retrieve by API.
    #No out put value, the df will be directly updated.
    #Don't forget to output the dataframe to csv

def Trending_Daily_Update(df,response):
    for i in range(len(response['items'])):
        try:
            likeCount = response['items'][i]['statistics']['likeCount']
        except:
            likeCount = None
        
        try:
            commentCount = response['items'][i]['statistics']['commentCount']
        except:
            commentCount = None
        
        try:
            favoriteCount = response['items'][i]['statistics']['favoriteCount']
        except:
            favoriteCount = None
            
        try:
            tags = response['items'][i]['snippet']['tags']
        except:
            tags = None
        
        df.loc[len(df.index)] = [response['items'][i]['id'],
                          response['items'][i]['snippet']['publishedAt'],
                          response['items'][i]['snippet']['title'],
                          response['items'][i]['snippet']['description'],
                          response['items'][i]['snippet']['channelId'],
                           response['items'][i]['snippet']['channelTitle'],
                           tags,
                           response['items'][i]['contentDetails']['dimension'],
                           response['items'][i]['contentDetails']['definition'],
                           response['items'][i]['contentDetails']['caption'],
                           response['items'][i]['contentDetails']['licensedContent'],
                           response['items'][i]['statistics']['viewCount'],
                           likeCount,
                           commentCount,
                           favoriteCount
                                ]
    if 'nextPageToken' in response.keys():
        nextPageToken = response['nextPageToken']
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            chart="mostPopular",
            regionCode="US",
            pageToken = response['nextPageToken']
            )
        
        response = request.execute()
        json_to_df(df,response)
        
    else:
        None

In [158]:
json_to_df(df,response)

In [159]:
df

,id,publish_time,title,description,channelId,channelTitle,tags,dimension,definition,caption,licensedContent,viewCount,likeCount,commentCount,favoriteCount
0,359IGLUlNhE,2022-11-24T15:49:57Z,Watch: Annual Macy's Thanksgiving Day parade,"In 1924, Macy's held its first parade but inst...",UCP6HGa63sBC7-KHtkme-p-g,USA TODAY,"[usa today, usa today news, macys, parade, tha...",2d,hd,false,True,1606606,9721,778,0
1,ml9eVc6ZR5A,2022-11-24T19:43:29Z,What Happened To Me,Why I Left YouTube....and am I back?,UC8rVYYeV6ekdAYIAYlaeTVA,LostNUnbound,"[LostNUnbound, LNU, What happened to LNU, Wher...",2d,hd,false,True,480725,58118,7686,0
2,kFGWYk1khMg,2022-11-25T02:01:13Z,Christmas With My Son!,Ho ho ho\n\nMerch: https://baylenlevine.com/\n...,UCwsjrSWqxLdcZrd9xVpmxog,Baylen Levine,"[baylen levine, baylen levine vlogs, benitez, ...",2d,hd,false,True,513649,33185,2720,0
3,359IGLUlNhE,2022-11-24T15:49:57Z,Watch: Annual Macy's Thanksgiving Day parade,"In 1924, Macy's held its first parade but inst...",UCP6HGa63sBC7-KHtkme-p-g,USA TODAY,"[usa today, usa today news, macys, parade, tha...",2d,hd,false,True,1606606,9721,778,0
4,ml9eVc6ZR5A,2022-11-24T19:43:29Z,What Happened To Me,Why I Left YouTube....and am I back?,UC8rVYYeV6ekdAYIAYlaeTVA,LostNUnbound,"[LostNUnbound, LNU, What happened to LNU, Wher...",2d,hd,false,True,480725,58118,7686,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,hI8wq7vUxoo,2022-11-19T16:57:35Z,Luka’s old school strategy that’s driving team...,Luka Doncic has added a new layer to his game ...,UC3HPbvB6f58X_7SMIp6OPYw,Thinking Basketball,"[NBA, NBA film analysis, NBA Analytics, Thinki...",2d,hd,false,True,486237,16667,1015,0
199,7IJPL8dtU_E,2022-11-20T03:59:31Z,Tennessee Volunteers vs. South Carolina Gameco...,South Carolina dominates No. 5 Tennessee at ho...,UCzRWWsFjqHk1an4OnVPsl9g,ESPN College Football,"[tennessee vs south carolina, tennessee, south...",2d,hd,false,True,366964,3761,1441,0
200,_VgeDBL_CLU,2022-11-19T17:38:12Z,Is this my last Video?,"Its been awhile, and in this video I explain ...",UChK8eDXz4le2_waQX5_5iuw,Rudys Adventure and Design,None,2d,hd,false,True,340888,29283,2086,0
201,Ke_MbvmDP0g,2022-11-20T15:00:11Z,My Ideal Thanksgiving Dinner,This is my ideal thanksgiving because you’ve a...,UChBEbMKI1eCcejTtmI32UEw,Joshua Weissman,"[sat bawl pro, joshua weissman, Thanksgiving, ...",2d,hd,false,True,969709,35966,1508,0


In [160]:
df.to_csv('day_one_trending_video.csv')

In [156]:
response['items'][3]

{'kind': 'youtube#video',
 'etag': 'P-0Uy9ZAK_KmIOtha1SlOPABtiM',
 'id': '1UiJQ3PNEaM',
 'snippet': {'publishedAt': '2022-11-25T00:25:08Z',
  'channelId': 'UCD6VugMZKRhSyzWEWA9W2fg',
  'title': 'Age of Wonders 1 Review | 800x600® Edition™',
  'description': 'Go to https://expressvpn.com/sseth and find \nout how you can get 3 months of \nExpressVPN free!\n\nGet Age of Wonders 1, 2 and 3 on GOG:\nhttps://af.gog.com/redeem/THUGHUNTER?as=1630110786\n\nMapViewer.ShowScene Crash fix:\nhttps://drive.google.com/file/d/1pGZbtVzmY1otLWe_2k4ytO_28vJdbyCK/view?usp=sharing\n(put in your install folder and run it)\n\nNo merchants were charged in the making of this video.\n\n-----------------------\nSend Sseth Shekels: https://www.paypal.me/SsethTzeentachGB\nSend Sseth Shekels per video:  https://www.patreon.com/Sseth\nSend Sseth Shekels / crypto: https://www.subscribestar.com/ssethtzeentach\n\nWebsite: https://www.ssethtzeentach.com/\nTwitter: https://twitter.com/SsethTzeentach',
  'thumbnails': {'d

In [151]:
request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    chart="mostPopular",
    regionCode="US"
)
response = request.execute()

print(response)


{'kind': 'youtube#videoListResponse', 'etag': 'DOysLSEr1J-sTgxftzWEaix4gvQ', 'items': [{'kind': 'youtube#video', 'etag': 'u7DE5aeq5WTwlQiB67mbjMWI4aA', 'id': '359IGLUlNhE', 'snippet': {'publishedAt': '2022-11-24T15:49:57Z', 'channelId': 'UCP6HGa63sBC7-KHtkme-p-g', 'title': "Watch: Annual Macy's Thanksgiving Day parade", 'description': "In 1924, Macy's held its first parade but instead of focusing on Thanksgiving it was called Macy's Christmas Parade. The parade was held just by store employees and the hope was the celebration would influence customers to shop for the coming Christmas holiday. \n\nRELATED: 95th annual Macy’s Thanksgiving Day Parade takes place in New York https://bit.ly/3grHNwK\n\n» Subscribe to USA TODAY: http://bit.ly/1xa3XAh\n» Watch more on this and other topics from USA TODAY: https://bit.ly/2IMPbAh\n» USA TODAY delivers current local and national news, sports, entertainment,  finance, technology, and more through award-winning journalism, photos, videos and VR.\n\